In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('./melb_data/melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
X_train.head()

,Rooms,Distance,Landsize,BuildingArea,YearBuilt
10702,3,8.8,436.0,NaN,NaN
1301,3,5.2,209.0,NaN,NaN
9113,2,2.0,108.0,162.0,2005.0
11997,2,19.6,214.0,115.0,2000.0
9928,3,7.8,103.0,120.0,1997.0


# Gradient Boosting

`Gradient boosting` is a method that goes through cycles to iteratively add models into an ensemble. It begins by initializing the ensemble with a single model, whose predictions can be pretty naive. (Even if its predictions are wildly inaccurate, subsequent additions to the ensemble will address those errors.)

1) First, we use the current ensemble to generate predictions for each observation in the dataset. To make a prediction, we add the predictions from all models in the ensemble.

2) These predictions are used to calculate a loss function (like mean squared error, for instance).

3) hen, we use the loss function to fit a new model that will be added to the ensemble. Specifically, we determine model parameters so that adding this new model to the ensemble will reduce the loss. (Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use gradient descent on the loss function to determine the parameters in this new model.)

4) Finally, we add the new model to ensemble, and repeat!

<img src = "https://i.imgur.com/MvCGENh.png">

<h3>XGBoost Library (Extreme Gradient Boosting)</h3>

It is an implementation of gradient boosting with several additional features focused on performance and speed. (Scikit-learn has another version of gradient boosting, but XGBoost has some technical advantages.)

In [6]:
from xgboost import XGBRegressor

In [8]:
xg_reg_model = XGBRegressor()
xg_reg_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [10]:
from sklearn.metrics import mean_absolute_error
y_pred = xg_reg_model.predict(X_valid)
mean_absolute_error(y_valid,y_pred)

236272.09369247055

<h3>Parameter Tuning</h3>
XGBoost has a few parameters that can dramatically affect accuracy and training speed. The first parameters you should understand are:

<b>n_estimators:</b>
It specifies how many times to go through the modeling cycle described above. It is equal to the number of models that we include in the ensemble.<br>
Too low a value causes `underfitting`, which leads to inaccurate predictions on both training data and test data.<br>
Too high a value causes `overfitting`, which causes accurate predictions on training data, but inaccurate predictions on test data (which is what we care about).<br>
Typical values range from 100-1000, though this depends a lot on the learning_rate parameter discussed below.

<b>early_stopping_rounds:</b>
It offers a way to automatically find the ideal value for n_estimators. Early stopping causes the model to stop iterating when the validation score stops improving, even if we aren't at the hard stop for n_estimators. `It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.` Since random chance sometimes causes a single round where validation scores don't improve, you need to `specify a number for how many rounds of straight deterioration to allow before stopping`. Setting early_stopping_rounds=5 is a reasonable choice. In this case, we stop after 5 straight rounds of deteriorating validation scores.

When using early_stopping_rounds, you also need to set aside some data for calculating the validation scores - this is done by setting the `eval_set parameter`.

<b>learning_rate:</b>
Instead of getting predictions by simply adding up the predictions from each component model, we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in.

This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without overfitting. If we use early stopping, the appropriate number of trees will be determined automatically.

In general, `a small learning rate and large number of estimators will yield more accurate XGBoost models, though it will also take the model longer to train` since it does more iterations through the cycle. As default, XGBoost sets learning_rate=0.1.

<b>n_jobs:</b>
On larger datasets where runtime is a consideration, you can use `parallelism` to build your models faster. It's common to `set the parameter n_jobs equal to the number of cores on your machine. On smaller datasets, this won't help.`

The resulting model won't be any better, so micro-optimizing for fitting time is typically nothing but a distraction. But, it's useful in large datasets where you would otherwise spend a long time waiting during the fit command.

In [20]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [21]:
y_pred = xg_reg_model.predict(X_valid)
mean_absolute_error(y_valid,y_pred)

235905.60242083948